In [1]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.com%2Foauth2callback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&

In [2]:
!sudo apt-get install -qq w3m # to act as web browser
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package w3m.
(Reading database ... 123074 files and directories currently installed.)
Preparing to unpack .../w3m_0.5.3-37ubuntu0.1_amd64.deb ...
Unpacking w3m (0.5.3-37ubuntu0.1) ...
Setting up w3m (0.5.3-37ubuntu0.1) ...
Processing triggers for man-db (2.9.1-1) ...
Processing triggers for mime-support (3.64ubuntu1) ...
/content
/content/drive
/content
/
Access token retrieved correctly.


In [3]:
import os
import tensorflow.keras as keras

data: dict
epochs_ = 300
lr = 0.001
kernel = (2,2)
size_images = (192, 256)

In [4]:
import numpy as np

def index_genre(genre, genres):
    for (g,index) in zip(genres, range(len(genres))):
        if(g == genre):
            return index
    return -1

def get_data(data_path, genres, decoder, training_percentaje=0.6, validation_percentaje=0.2, test_percentaje=0.2):
    """
    data_path: se le pasa la direccion de la carpeta donde se encuentra la base de datos.
    genres: se le pasa una lista con los nombres da cada carpeta que contiene un genero dado.
    decoder: funcion para decodificar el dato que se le pasa, por ejemplo en caso de imagenes habria hacer imread
    """

    data_training = {'in': [], 'out': []}
    data_validation = {'in': [], 'out': []}
    data_test = {'in': [], 'out': []}

    for genre in genres:
        files = os.listdir(data_path + genre)
        count = len(files)

        for (filename, index) in zip(files, range(len(files))):
            filepath = data_path + genre + '/' + filename

            if (index < training_percentaje * count):
                data_training['in'].append(decoder(filepath))
                data_training['out'].append(index_genre(genre, genres))
                continue

            if (index < (training_percentaje + validation_percentaje) * count):
                data_validation['in'].append(decoder(filepath))
                data_validation['out'].append(index_genre(genre, genres))
            else:
                data_test['in'].append(decoder(filepath))
                data_test['out'].append(index_genre(genre, genres))

    data_training = {'in': np.array(data_training['in']),'out': np.array(data_training['out'])}
    data_validation = {'in': np.array(data_validation['in']),'out': np.array(data_validation['out'])}
    data_test = {'in': np.array(data_test['in']),'out': np.array(data_test['out'])}

    return {
        'data_training': data_training,
        'data_validation': data_validation,
        'data_testing': data_test
    }

In [5]:
import cv2

def read_image(filepath):
    global size_images
    image = cv2.imread(filepath)
    return cv2.resize(image, size_images)

def read_gray_image(filepath):
    global size_images
    img_gray = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

    # Convierte la matriz de la imagen a un tipo de datos float32
    # img_gris = img_gris.astype('float32')

    # Agrega una dimensión adicional parael canal de la imagen
    img_gray = np.expand_dims(img_gray, axis=-1)

    return cv2.resize(img_gray, size_images)

genres = ['blues', 'classical', 'country', 'disco',
          'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

data_path = '/content/drive/MyDrive/MFCC_dataset/'

def get_data_mfcc(training_percentaje=0.6, validation_percentaje=0.2, test_percentaje=0.2):
    return get_data(data_path, genres, read_image,training_percentaje,validation_percentaje,test_percentaje)

In [9]:
def data_internal():
    global data
    data = get_data_mfcc(training_percentaje=0.7, validation_percentaje=0.15, test_percentaje=0.15)

def training():
    global data
    global epochs_
    global size_images
    global lr

    data_internal()
    training_data = data['data_training']
    v_data = data['data_validation']

    model = mfcc_cnn_model(input_shape=(
        size_images[1], size_images[0], 3), classes=10, filter_shape=kernel)

    model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(),
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        metrics=['accuracy'])

    checkpoint = keras.callbacks.ModelCheckpoint(
        '/content/drive/MyDrive/mfcc_models/best_weights.h5',
        save_weights_only=True,
        save_best_only=True,
        monitor='val_accuracy',
        mode='max')

    model.fit(
        training_data['in'], training_data['out'],
        validation_data=(v_data['in'], v_data['out']),
        epochs=epochs_,
        callbacks=[checkpoint]
        )

    model.save('/content/drive/MyDrive/mfcc_models/mfcc_model.h5')

def testing(model_name="mfcc_model.h5", epoch_name = 'best_weights.h5'):
    global data
    global size_images

    # data_internal()
    input = data['data_testing']['in']
    output = data['data_testing']['out']

    path = '/content/drive/MyDrive/mfcc_models/' + model_name
    path_best_epoch = '/content/drive/MyDrive/mfcc_models/' + epoch_name

    model = keras.models.load_model(path)
    # model.load_weights(path_best_epoch)

    score = model.evaluate(input, output, verbose=0)
    # print('testing loss: ' + str(score[0]))
    print('testing accuracy: ' + str(score[1]))

    input = data['data_validation']['in']
    output = data['data_validation']['out']
    score = model.evaluate(input, output, verbose=0)
    # print('validation loss: ' + str(score[0]))
    print('validation accuracy: ' + str(score[1]))

    input = data['data_training']['in']
    output = data['data_training']['out']
    score = model.evaluate(input, output, verbose=0)
    # print('training loss: ' + str(score[0]))
    print('training accuracy: ' + str(score[1]))


In [7]:
def mfcc_cnn_model(input_shape=(128, 96, 1), classes=10, filter_shape=(3, 3)):
    X_input = keras.layers.Input(input_shape)

    X = keras.layers.Conv2D(32, filter_shape, activation='relu')(X_input)

    X = keras.layers.Conv2D(64, filter_shape, activation='relu')(X_input)
    X = keras.layers.AveragePooling2D((2, 2))(X)

    X = keras.layers.Conv2D(128, filter_shape, activation='relu')(X)
    X = keras.layers.AveragePooling2D((2, 2))(X)

    X = keras.layers.Conv2D(256, filter_shape, activation='relu')(X)
    X = keras.layers.GlobalAveragePooling2D()(X)

    X = keras.layers.Dense(256, activation='relu')(X)
    X = keras.layers.Dense(128, activation='relu')(X)
    X = keras.layers.Dense(64, activation='relu')(X)
    X = keras.layers.Dense(32, activation='relu')(X)
    X = keras.layers.Dense(classes, activation='softmax')(X)

    return keras.models.Model(inputs=X_input, outputs=X, name='SpectrogramCNN')

In [14]:
training()

Epoch 1/300
22/22 [==============================] - 10s 308ms/step - loss: 6.7268 - accuracy: 0.1017 - val_loss: 2.4751 - val_accuracy: 0.1000
Epoch 2/300
22/22 [==============================] - 2s 81ms/step - loss: 2.4287 - accuracy: 0.0931 - val_loss: 2.5082 - val_accuracy: 0.1000
Epoch 3/300
22/22 [==============================] - 2s 83ms/step - loss: 2.4822 - accuracy: 0.1074 - val_loss: 2.4638 - val_accuracy: 0.1000
Epoch 4/300
22/22 [==============================] - 4s 168ms/step - loss: 2.3477 - accuracy: 0.1175 - val_loss: 2.3233 - val_accuracy: 0.2067
Epoch 5/300
22/22 [==============================] - 2s 82ms/step - loss: 2.2548 - accuracy: 0.1590 - val_loss: 2.2701 - val_accuracy: 0.1533
Epoch 6/300
22/22 [==============================] - 2s 79ms/step - loss: 2.2253 - accuracy: 0.1991 - val_loss: 2.2168 - val_accuracy: 0.1667
Epoch 7/300
22/22 [==============================] - 2s 82ms/step - loss: 2.1738 - accuracy: 0.1977 - val_loss: 2.1983 - val_accuracy: 0.1800
Epo

In [13]:
#Save Weights in Model
def save_model(model_name="mfcc_model.h5", epoch_name = 'best_weights.h5'):
    path = '/content/drive/MyDrive/mfcc_models/' + model_name
    path_best_epoch = '/content/drive/MyDrive/mfcc_models/' + epoch_name
    model = keras.models.load_model(path)
    model.load_weights(path_best_epoch)
    model.save('/content/drive/MyDrive/mfcc_models/mfcc_model.h5')


In [16]:
save_model()
testing()

testing accuracy: 0.7986577153205872
validation accuracy: 0.7266666889190674
training accuracy: 0.9527220726013184
